<a href="https://colab.research.google.com/github/Vishsubb/futurecode/blob/main/PodcastGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive to store the pdf files for which we want to generate podcasts

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install pypdf libraries to load PDF and covert the same to text. Can skip this step if you already have text files


In [3]:
## Installing the necessary library to work with PDF
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.5/249.5 kB 5.6 MB/s eta 0:00:00


In [4]:
import requests
import io
from pypdf import PdfReader, PageRange
reader = PdfReader("/content/drive/MyDrive/Learning/Upskilling Blog - 1.pdf")

# Listing the number of pages in the PDF file
number_of_pages = len(reader.pages)
print (number_of_pages)

1


In [5]:
# Load only selected pages
pages = PageRange('0:5').to_slice()
selected_pages = reader.pages[pages]

# Extract text and store in pdf_input

In [8]:
pdf_input = ""
for page in selected_pages:
    pdf_input += page.extract_text() + "\n"

In [9]:
pdf_input

' \n \n \n \n \nEducational t echnology has enormous potential in upskilling a future -ready workforce. Let’s understand how tech \nis facilitating and augmenting  learning for a future -ready workforce:  \n• Upskill platforms are already tech-first:  Software platforms for upskilling need integration of  multiple \nstandalone technology platforms to provide an end -to-end learning experience. These may be course \nstorefronts, CRM systems, or course enrolment systems that include payment gateway solutions, teaching and \nlearning systems, assessments, and engage ment platforms. The interplay of technologies here helps in  \nattracting learners, enhancing the teaching & learning experience, and influencing learner outcomes.  \n \n• Driving down the cost of learner acquisition : Companies in the upskilling business are still face d with steep  \ncosts of learner acquisition . Tech  intervention can help a prospective learner better evaluate several learning \nplatforms with a  self-driv

# Determining tokens size. This step is done to determine the number of chunks of text that can be passed to ChatCompletion API taking into consideration size of the Prompt 

In [10]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.4 MB/s eta 0:00:00


In [11]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
token_list = enc.encode(pdf_input)

In [ ]:
token_list

In [ ]:
for token in token_list:
  print ("Token ", str(token), " decodes to ", enc.decode([token]))

In [14]:
## Please the variable as pdf_input, wiki_input, or book_input based on your choice
print ("Number of tokens in corpus ", len(enc.encode(pdf_input)))

Number of tokens in corpus  435


# Extract sentences. Create chunks to the closest fully completed sentences

In [15]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def split_text (input_text):
  #split_texts = sent_tokenize(input_text)
  #The sent_tokenize function  tokenize inserted text into sentences
  split_texts = sent_tokenize(input_text)
  return split_texts

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
def create_chunks(split_sents, max_token_len=2000):
  current_token_len = 0
  input_chunks = []
  current_chunk = ""
  for sents in split_sents:
    sent_token_len = len(enc.encode(sents))
    if (current_token_len + sent_token_len) > max_token_len:
      input_chunks.append(current_chunk)
      current_chunk = ""
      current_token_len = 0
    current_chunk = current_chunk + sents
    current_token_len = current_token_len + sent_token_len
  if current_chunk != "":
    input_chunks.append(current_chunk)
  return input_chunks

In [17]:
# split_sents = split_text(wiki_input)
# input_chunks = create_chunks(split_sents, max_token_len=1000)

split_sents = split_text(pdf_input)
input_chunks = create_chunks(split_sents, max_token_len=1000)

In [18]:
len(input_chunks)

1

# Generating the context of the podcast and give introduction text



**Initiate openai**

In [31]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**get openai key**

In [32]:
import openai
from getpass import getpass

openai.api_key = getpass('Enter your OPENAI_API_KEY  ')

Enter your OPENAI_API_KEY  ··········


In [ ]:
# Cheking of the API works
# we will be using the gpt-3.5-turbo version for this project
models = openai.Model.list()
for model in models["data"]:
  print (model["root"])

In [34]:
instructPrompt = """ContextText: """

In [35]:
systemPrompt = """Please summarize the text given in the ContextText as an abstract"""

**Add string ContextText before each chunk of text**

In [36]:
requestMessages = []
for text in input_chunks:
  requestMessage = instructPrompt + '\n' + text
  requestMessages.append(requestMessage)

**Get Summary for each chunk of text and add to list**

In [37]:
summaryOutputs = []
for request in requestMessages:
  summaryOutput = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                            messages=[{"role": "system", "content": systemPrompt},
                                                      {"role": "user", "content": request}
                                                      ]
                                            )
  summaryOutputs.append(summaryOutput)

**Concatenate all the summary strings**

In [38]:
summaryFacts = ""
for summaryfact in summaryOutputs:
  summaryFacts = summaryFacts + summaryfact.choices[0].message.content

In [39]:
summaryFacts

"The use of technology in education has the potential to upskill a future-ready workforce. Software platforms for upskilling integrate multiple technology platforms to provide an end-to-end learning experience, including course storefronts, CRM systems, or course enrollment systems with payment gateway solutions, teaching and learning systems, assessments, and engagement platforms. Technology can also reduce the cost of learner acquisition by allowing prospective learners to evaluate different learning platforms with a self-driven approach, providing a learner portal that facilitates testing the learning material, evaluating ROI, and providing flexibility in class schedules. Additionally, technology-driven skills mapping and gap analysis can shape personalized career paths by assessing learners' skills, identifying gaps and devising transformational paths for upskilling or reskilling."

#For the summary thus created. create an introduction starting with interesting questions and welcome the audience and guest

In [58]:
instructPrompt+ summaryFacts

"ContextText: The use of technology in education has the potential to upskill a future-ready workforce. Software platforms for upskilling integrate multiple technology platforms to provide an end-to-end learning experience, including course storefronts, CRM systems, or course enrollment systems with payment gateway solutions, teaching and learning systems, assessments, and engagement platforms. Technology can also reduce the cost of learner acquisition by allowing prospective learners to evaluate different learning platforms with a self-driven approach, providing a learner portal that facilitates testing the learning material, evaluating ROI, and providing flexibility in class schedules. Additionally, technology-driven skills mapping and gap analysis can shape personalized career paths by assessing learners' skills, identifying gaps and devising transformational paths for upskilling or reskilling."

In [59]:
systemPrompt = """You are an automated podcast script generator. Input is a text given as ContextText. Host name is Vish and guest name is Dr. Sheetal. You have 3 activities to perform. 
First activity should be prefixed Vish. Second activity should be prefixed Vish. Third activity to be prefixed with name Dr. Sheetal
First Activity: Host to welcome the audience, introduce oneself to new audience. 
Second Activity: Using key aspects in the text, generate introduction to podcast starting with 3 interesting questions. Keep the introduction to 30 words. Welcome the guest to the show. 
Third Activity: for guest, generate a response thanking the host for having them in the show. Do not output activity names"""

In [60]:
 introductionOutput = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                            messages=[{"role": "system", "content": systemPrompt},
                                                      {"role": "user", "content":instructPrompt+ summaryFacts}
                                                      ]
                                                  )


In [61]:
podcastIntroText = introductionOutput.choices[0].message.content

In [62]:
podcastIntroText

"Vish: Hello and welcome everyone to our show. This is Vish, and I am excited to have you all here. For those of you who are new, I am a host and content creator who loves to have insightful discussions with experts. \n\nVish: Today, we have Dr. Sheetal with us to talk about the potential of technology in education. We'll be exploring how software platforms can provide an end-to-end learning experience, reduce learner acquisition costs, and personalize career paths. \n\nVish: Dr. Sheetal, we're thrilled to have you on the show. To kick things off, can you share with us: \n\n- How can technology help learners evaluate different learning platforms?\n- What are the benefits of using technology for personalized career paths?\n- Can you tell us more about technology-driven skills mapping and gap analysis?\n\nDr. Sheetal: Vish, thank you for having me on the show. I'm excited to be here and share my insights with your audience. Regarding your questions, technology can provide a self-driven a

The above output should be a list of all important facts, themes or other highlights from your topic that you would like to be covered in the podcast conversation

#Generating the actual podcast questions and answers



In [63]:
instructPrompt = """ContextText: """

In [64]:
systemPrompt = """You are an automated podcast script generator. 
Input is a text given as ContextText. 
Host name is Vish and guest name is Dr. Sheetal. 
Using key aspects in the text, generate 3 unique and interesting questions.  
For generated question ,generate corresponding answers. Limit answers to 50 words.  
Always make sure, the questions are prefixed with the Vish and Answers are prefixed with Dr. Sheetal.  
Keep the discussion conversational and engaging. Do not introduce or welcome the guest. Do not number the questions"""

In [65]:
requestMessages = []
for text in input_chunks:
  requestMessage = instructPrompt + '\n' + text
  requestMessages.append(requestMessage)

In [67]:
chatOutputs = []
for request in requestMessages:
  
  chatOutput = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                            messages=[{"role": "system", "content": systemPrompt},
                                                      {"role": "user", "content": request}
                                                      ]
                                            )
  chatOutputs.append(chatOutput)

In [68]:
podcastFacts = ""
for chats in chatOutputs:
  podcastFacts = podcastFacts + chats.choices[0].message.content +"\n"

In [69]:
podcastFacts

"Vish: How can technology help in reducing the cost of learner acquisition in the upskilling business?\nDr. Sheetal: Technology can provide a self-driven approach for prospective learners to evaluate learning platforms, reducing the cost of customer acquisition. Learner portals equipped with toolsets to evaluate ROI, fast-track completion, and flexible class schedules can provide a feel of the course before enrolment, helping learners make a well-informed decision.\n\nVish: How can personalized career paths be shaped for the learners using educational technology?\nDr. Sheetal: Tech-driven skill mapping and gap analysis can maintain a skills repository and identify gaps in learners' skills and chalk out a personalized path for upskilling or reskilling. Initial assessments and identification of learners' skills paired with a mechanism for transformational paths helps them shift to aspirational roles.\n\nVish: Could you elaborate on how integration of multiple standalone technology platfo

# Generating ending and final output



In [76]:
systemPrompt = """You are an automated podcast script generator. 
Input is a text given as ContextText. 
Host name is Vish and guest name is Dr. Sheetal. You need to perform 3 activities. 
First Activity needs to be prefixed with Vish. Second activity needs to be prefixed with Dr. Sheetal. Third Activity needs to be prefixed with Vish. 
First Activity: Using key aspects in the text, generate closing remarks of podcast, summarizing outcomes. 
Second Activity: Guest needs to appreciate the host for the summary, thank the host and audience. 
Third Activity: Host to sign off professionally, bidding thank you to the guest and looking forward to audience participation for future podcasts.
Do not output activity names. Keep the output conversational"""
podcastPrompt = """ContextText:"""

In [71]:
requestMessage = podcastPrompt + summaryFacts
requestMessage

"ContextText:The use of technology in education has the potential to upskill a future-ready workforce. Software platforms for upskilling integrate multiple technology platforms to provide an end-to-end learning experience, including course storefronts, CRM systems, or course enrollment systems with payment gateway solutions, teaching and learning systems, assessments, and engagement platforms. Technology can also reduce the cost of learner acquisition by allowing prospective learners to evaluate different learning platforms with a self-driven approach, providing a learner portal that facilitates testing the learning material, evaluating ROI, and providing flexibility in class schedules. Additionally, technology-driven skills mapping and gap analysis can shape personalized career paths by assessing learners' skills, identifying gaps and devising transformational paths for upskilling or reskilling."

In [72]:
finalOutput = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                            messages=[{"role": "system", "content": systemPrompt},
                                                      {"role": "user", "content": requestMessage}
                                                      ]
                                            )

In [73]:
podcastclosingScript = finalOutput.choices[0].message.content

In [74]:
podcastclosingScript

'The potential of technology in education and upskilling has been highlighted in our discussion with Dr. Sheetal. We learned about the integration of multiple technology platforms for an end-to-end learning experience, which can reduce the cost of learner acquisition and shape personalized career paths. With technology-driven skills mapping and gap analysis, learners can devise transformational paths for upskilling and reskilling. This discussion emphasizes the crucial role of technology in preparing a future-ready workforce. \n\nDr. Sheetal appreciates the summary and would like to thank Vish and the audience for the opportunity to discuss this important topic. She believes that technology can enable education to be more accessible and inclusive, and hopes that this discussion inspires learners to take advantage of the opportunities available through technology. \n\nWe thank Dr. Sheetal for sharing her insights on the potential of technology in education and upskilling. We urge our au

In [75]:
podcastScript = podcastIntroText+  podcastFacts + podcastclosingScript
podcastScript

"Vish: Hello and welcome everyone to our show. This is Vish, and I am excited to have you all here. For those of you who are new, I am a host and content creator who loves to have insightful discussions with experts. \n\nVish: Today, we have Dr. Sheetal with us to talk about the potential of technology in education. We'll be exploring how software platforms can provide an end-to-end learning experience, reduce learner acquisition costs, and personalize career paths. \n\nVish: Dr. Sheetal, we're thrilled to have you on the show. To kick things off, can you share with us: \n\n- How can technology help learners evaluate different learning platforms?\n- What are the benefits of using technology for personalized career paths?\n- Can you tell us more about technology-driven skills mapping and gap analysis?\n\nDr. Sheetal: Vish, thank you for having me on the show. I'm excited to be here and share my insights with your audience. Regarding your questions, technology can provide a self-driven a

Check the output script that your prompt generated and make necessary changes if required. Prompt design is iterative in nature and don't be shy to explore different styles.

# Section 4 - Generating the voice and putting the podcast together


In [46]:
!pip install elevenlabs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00


In [47]:
from elevenlabs import set_api_key

set_api_key(getpass('Enter your ELEVEN_LABS_API_KEY  '))

Enter your ELEVEN_LABS_API_KEY  ··········


In [48]:
from elevenlabs import generate

def createPodcast(podcastScript, speakerName1, speakerChoice1, speakerName2, speakerChoice2):
  genPodcast = []
  podcastLines = podcastScript.split('\n\n')
  podcastLineNumber = 0
  for line in podcastLines:
    
    if podcastLineNumber % 2 == 0:
      speakerChoice = speakerChoice1
      line = line.replace(speakerName1+":", '')
    else:
      speakerChoice = speakerChoice2
      line = line.replace(speakerName2+":", '')
    genVoice = generate(text=line, voice=speakerChoice, model="eleven_monolingual_v1")
    genPodcast.append(genVoice)
    podcastLineNumber += 1
  return genPodcast

In [ ]:
podcastLines = podcastScript.split('\n\n')
genPodcast = []
for line in podcastLines:
  print(line)
  if (line.strip().startswith("Vish") == True):
    speakerChoice = "Sam"
    line = line.replace("Vish:", '')
  else:
    speakerChoice = "Bella"
    line = line.strip().replace("Dr. Sheetal:", '') 
   genVoice = generate(text=line, voice=speakerChoice, model="eleven_monolingual_v1")
   genPodcast.append(genVoice)


In [ ]:
speakerName1 = "Vish"
speakerChoice1 = "Sam"
speakerName2 = "Dr. Sheetal"
speakerChoice2 = "Bella"
genPodcast = createPodcast(podcastScript, speakerName1, speakerChoice1, speakerName2, speakerChoice2)

In [ ]:
type(genPodcast)

list

In [56]:
with open("/content/drive/MyDrive/Learning/EDTech Blog Podcast.mpeg", "wb") as f:
  for pod in genPodcast:
    f.write(pod)